<a href="https://colab.research.google.com/github/jaemmy425/Jaemmy425.github.io/blob/main/%E8%B4%A8%E5%BF%83%E8%B7%9D%E7%A6%BB%E8%AE%A1%E7%AE%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import os
import math
import pandas as pd
import numpy as np

In [26]:
# 用户文件路径
file_a = r"/content/OS_CB_centroid.xls"
file_b = r"/content/OSM_CB_centroid.xls"
def read_centroids(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"找不到文件: {path}")
    try:
        df = pd.read_excel(path)
    except Exception as e:
        # 提示常见读取问题
        raise RuntimeError(f"读取 Excel 文件失败: {path}\n错误: {e}")
    return df


df_a = read_centroids(file_a)
df_b = read_centroids(file_b)

In [27]:
# 标准化列名，方便后续处理（保留原始列名的副本）
for df in (df_a, df_b):
    df.columns = [c.strip() if isinstance(c, str) else c for c in df.columns]

# 确保必要列存在
required_cols = ['LSOA21NM', 'XCoord', 'YCoord']
for name, df in [('OS', df_a), ('OSM', df_b)]:
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        raise KeyError(f"文件 {name} 缺少必要列: {missing}")

# 规范 LSOA21NM 字段用于合并（小写并去空白）
df_a['_LSOA21NM_key'] = df_a['LSOA21NM'].astype(str).str.strip().str.lower()
df_b['_LSOA21NM_key'] = df_b['LSOA21NM'].astype(str).str.strip().str.lower()

# 确保 XCoord/YCoord 为数值，不能为 NaN
for df, name in [(df_a, 'OS'), (df_b, 'OSM')]:
    df['XCoord_num'] = pd.to_numeric(df['XCoord'], errors='coerce')
    df['YCoord_num'] = pd.to_numeric(df['YCoord'], errors='coerce')
    n_missing = df['XCoord_num'].isna().sum() + df['YCoord_num'].isna().sum()
    if n_missing>0:
        print(f"警告: 文件 {name} 存在 {n_missing} 个 X/Y 无法解析为数值（将导致这些行被标记为无法计算距离）")

In [28]:
# 确保 XCoord/YCoord 为数值，不能为 NaN
for df, name in [(df_a, 'OS'), (df_b, 'OSM')]:
    df['XCoord_num'] = pd.to_numeric(df['XCoord'], errors='coerce')
    df['YCoord_num'] = pd.to_numeric(df['YCoord'], errors='coerce')
    n_missing = df['XCoord_num'].isna().sum() + df['YCoord_num'].isna().sum()
    if n_missing>0:
        print(f"警告: 文件 {name} 存在 {n_missing} 个 X/Y 无法解析为数值（将导致这些行被标记为无法计算距离）")

In [29]:
# 按 LSOA21NM 合并（内连接以得到两表都有的 LSOA21NM）
merged = pd.merge(df_a, df_b, on='_LSOA21NM_key', how='outer', suffixes=('_OS','_OSM'), indicator=True)

# 计算距离（如果任一坐标缺失，结果为 NaN）

def compute_distance(row):
    x1 = row.get('XCoord_num_OS')
    y1 = row.get('YCoord_num_OS')
    x2 = row.get('XCoord_num_OSM')
    y2 = row.get('YCoord_num_OSM')
    if pd.isna(x1) or pd.isna(y1) or pd.isna(x2) or pd.isna(y2):
        return np.nan
    dx = float(x2) - float(x1)
    dy = float(y2) - float(y1)
    return math.hypot(dx, dy)  # 投影坐标系下的欧氏距离（米）

# 为了计算方便，先把 numeric 列重命名到合并表中
merged = merged.rename(columns={
    'XCoord_num_OS': 'XCoord_num_OS',
    'YCoord_num_OS': 'YCoord_num_OS',
    'XCoord_num_OSM': 'XCoord_num_OSM',
    'YCoord_num_OSM': 'YCoord_num_OSM'
})

# 如果合并后 numeric 列名不是上面的形式，尝试从原始列推导
for col in ['XCoord_num_OS','YCoord_num_OS','XCoord_num_OSM','YCoord_num_OSM']:
    if col not in merged.columns:
        # 尝试找类似的列
        possibles = [c for c in merged.columns if c.lower().startswith(col.split('_')[0].lower())]
        # 不做强制，留给后续计算尝试

# 先尝试把原始 numeric 列合并进来（在 merge 时 pandas 已把列名后缀合并）
if 'XCoord_num_OS' not in merged.columns and 'XCoord_num' in df_a.columns:
    merged['XCoord_num_OS'] = merged['XCoord_num']
if 'YCoord_num_OS' not in merged.columns and 'YCoord_num' in df_a.columns:
    merged['YCoord_num_OS'] = merged['YCoord_num']
if 'XCoord_num_OSM' not in merged.columns and 'XCoord_num' in df_b.columns:
    # 合并后会把两个相同名的列变成 XCoord_num_x / XCoord_num_y，尝试处理这些情况
    if 'XCoord_num_x' in merged.columns:
        merged['XCoord_num_OS'] = merged['XCoord_num_x']
    if 'XCoord_num_y' in merged.columns:
        merged['XCoord_num_OSM'] = merged['XCoord_num_y']
    if 'YCoord_num_x' in merged.columns:
        merged['YCoord_num_OS'] = merged['YCoord_num_x']
    if 'YCoord_num_y' in merged.columns:
        merged['YCoord_num_OSM'] = merged['YCoord_num_y']

# 通用方式：如果列没有按预期命名，尝试根据后缀寻找
for base in ['XCoord_num','YCoord_num']:
    if base + '_OS' not in merged.columns:
        for c in merged.columns:
            if c.lower().startswith(base.lower()) and c.endswith('_x'):
                merged[base + '_OS'] = merged[c]
    if base + '_OSM' not in merged.columns:
        for c in merged.columns:
            if c.lower().startswith(base.lower()) and c.endswith('_y'):
                merged[base + '_OSM'] = merged[c]

# 现在计算距离
merged['distance_m'] = merged.apply(compute_distance, axis=1)

# 统计匹配情况
n_total = len(merged)
ngood = merged['distance_m'].notna().sum()
only_a = (merged['_merge']=='left_only').sum()
only_b = (merged['_merge']=='right_only').sum()
print(f"总 LSOA21NM 数: {n_total}, 双表都有的数量(可计算距离): {ngood}, 仅在 OS: {only_a}, 仅在 OSM: {only_b}")

# 保存输出
out_path = r"CB_Centroid_distances.xlsx"
try:
    merged.to_excel(out_path, index=False)
    print(f"已保存结果到: {out_path}")
except Exception as e:
    print(f"保存文件失败: {e}")

# 如果需要，只显示前几行
print(merged[['LSOA21NM_OS','LSOA21NM_OSM','_LSOA21NM_key','XCoord_num_OS','YCoord_num_OS','XCoord_num_OSM','YCoord_num_OSM','distance_m']].head(10))

总 LSOA21NM 数: 84, 双表都有的数量(可计算距离): 80, 仅在 OS: 4, 仅在 OSM: 0
已保存结果到: CB_Centroid_distances.xlsx
      LSOA21NM_OS    LSOA21NM_OSM   _LSOA21NM_key  XCoord_num_OS  \
0  Cambridge 001A  Cambridge 001A  cambridge 001a  545734.852973   
1  Cambridge 001B  Cambridge 001B  cambridge 001b  546416.221757   
2  Cambridge 001C  Cambridge 001C  cambridge 001c  546095.331571   
3  Cambridge 001D  Cambridge 001D  cambridge 001d  545309.258388   
4  Cambridge 001E  Cambridge 001E  cambridge 001e  545282.181703   
5  Cambridge 001F  Cambridge 001F  cambridge 001f  545873.470435   
6  Cambridge 002A  Cambridge 002A  cambridge 002a  544471.477858   
7  Cambridge 002B  Cambridge 002B  cambridge 002b  544666.600077   
8  Cambridge 002C  Cambridge 002C  cambridge 002c  544442.786019   
9  Cambridge 002D  Cambridge 002D  cambridge 002d  544883.085171   

   YCoord_num_OS  XCoord_num_OSM  YCoord_num_OSM  distance_m  
0  261160.755378     545786.7500   261109.703125   72.798585  
1  261015.620416     546403.9375